In [8]:
%pip install --upgrade --quiet docx2txt langchain-community
%pip install -qU langchain-text-splitters langchain-chroma langchain-core langchain-upstage
%pip install -U langchain langchainhub --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


- Chroma : Vector in-memory Database

In [1]:
%pip install -qU langchain langchain-pinecone langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [6]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
from langchain.chains import RetrievalQA
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

load_dotenv()
query = '연봉 5천만원 초과인 직장인의 소득세는 얼마인가요?'

embedding = OpenAIEmbeddings(model="text-embedding-3-large")
index_name = "tax-index"
database = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embedding)

llm = ChatOpenAI(model='gpt-4o')
prompt = hub.pull("rlm/rag-prompt")
retriever = database.as_retriever(search_kwargs={'k': 4})

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = retriever,
    chain_type_kwargs={"prompt" : prompt}
)

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt = ChatPromptTemplate.from_template(f"""
사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
그런 경우에, 질문만 답변해주세요.

사전 : {dictionary}

질문 : {{question}}

""")

dictionary_chain = prompt | llm | StrOutputParser()
tax_chain = {"query" : dictionary_chain} | qa_chain
ai_response = tax_chain.invoke({"question": query})

In [8]:
ai_response['result']

'연봉이 5천만원을 초과하는 거주자의 소득세는 기본적으로 624만원에다가 5천만원을 초과하는 금액의 24%를 더한 금액입니다. 구체적인 소득 금액에 따라 계산이 필요합니다.'

In [9]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrived_docs = database.similarity_search(query, k = 3)
print(retrived_docs)

[Document(id='108838a5-8d68-4770-8ad8-f50c8041090b', metadata={'source': './tax_with_markdown.docx'}, page_content='[전문개정 2009. 12. 31.]\n\n[제목개정 2014. 1. 1.]\n\n\n\n제4절 세액의 계산 <개정 2009. 12. 31.>\n\n\n\n제1관 세율 <개정 2009. 12. 31.>\n\n\n\n제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 